In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [38]:
import pandas as pd
import numpy as np
import torch
# from transformers import BertTokenizer, BertForSequenceClassification
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

In [52]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [53]:
'''
# customer_country, customer_country.1 날리고 response_orporate만 남기기
'''
drop_col = ['customer_country.1', 'customer_country']
df_train.drop(drop_col, axis=1, inplace=True)
df_test.drop(drop_col, axis=1, inplace=True)

In [54]:
label_columns = list(df_train.columns)

label_columns.remove("is_converted")

In [55]:
non_null_data = df_train.dropna(subset=['ver_win_ratio_per_bu'])

# business_area에 따른 ver_win_ratio_per_bu의 평균 계산
mean_by_area = non_null_data.groupby('business_area')['ver_win_ratio_per_bu'].mean().reset_index()

# business_unit에 따른 ver_win_ratio_per_bu의 평균 계산
mean_by_unit = non_null_data.groupby('business_unit')['ver_win_ratio_per_bu'].mean().reset_index()

# 결측치를 채우는 함수 정의
def fill_ver_win_ratio_per_bu(row):
    if pd.isnull(row['ver_win_ratio_per_bu']):
        # 결측치인 경우 해당 business_unit의 평균값으로 채우기
        business_unit = row['business_unit']
        if not pd.isnull(business_unit):
            mean_value = mean_by_unit[mean_by_unit['business_unit'] == business_unit]['ver_win_ratio_per_bu'].values
            if len(mean_value) > 0:
                return mean_value[0]
    return row['ver_win_ratio_per_bu']

# 결측치 처리 함수를 적용하여 ver_win_ratio_per_bu 열의 결측치 채우기
df_train['ver_win_ratio_per_bu'] = df_train.apply(fill_ver_win_ratio_per_bu, axis=1)
df_test['ver_win_ratio_per_bu'] = df_test.apply(fill_ver_win_ratio_per_bu, axis=1)

global_mean_ver_win_ratio_per_bu = df_train['ver_win_ratio_per_bu'].mean()

# 나머지 결측치를 전체 평균값으로 채우기
df_train['ver_win_ratio_per_bu'].fillna(global_mean_ver_win_ratio_per_bu, inplace=True)
df_test['ver_win_ratio_per_bu'].fillna(global_mean_ver_win_ratio_per_bu, inplace=True)


In [56]:
'''
# 수치형 변수 전체 Standardizaiton, Normalization 실행
'''
# Initialize scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
columns_to_ST = ["com_reg_ver_win_rate", "historical_existing_cnt","lead_desc_length"]
columns_to_NM = ["ver_win_rate_x", "ver_win_ratio_per_bu"]

# Apply Standardization to each column
for column in columns_to_ST:
    # Standardization
    df_train[column + '_standardized'] = scaler_standard.fit_transform(df_train[[column]])
    df_test[column + '_standardized'] = scaler_standard.fit_transform(df_test[[column]])
    # Drop the original column
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

# Apply Standardization to each column
for column in columns_to_NM:
    # Normalization
    df_train[column + '_normalized'] = scaler_minmax.fit_transform(df_train[[column]])
    df_test[column + '_normalized'] = scaler_minmax.fit_transform(df_test[[column]])
    # Drop the original column
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

# Print the first few rows of the DataFrame to check the results
print(df_train.head())

   bant_submit business_unit  customer_idx          customer_type  enterprise  \
0          1.0            AS         32160           End-Customer  Enterprise   
1          1.0            AS         23122           End-Customer  Enterprise   
2          1.0            AS          1755           End-Customer  Enterprise   
3          1.0            AS          4919           End-Customer  Enterprise   
4          1.0            AS         17126  Specifier/ Influencer  Enterprise   

   id_strategic_ver  it_strategic_ver  idit_strategic_ver  \
0               NaN               NaN                 NaN   
1               NaN               NaN                 NaN   
2               NaN               NaN                 NaN   
3               NaN               NaN                 NaN   
4               NaN               NaN                 NaN   

              customer_job                        inquiry_type  ... ver_pro  \
0               purchasing  Quotation or purchase consultation  ...

In [57]:
'''
# business_area 날리고 2개 피처 추가
'''
def is_hospital(value):
    if value == "hospital & health care":
        return 1
    else:
        return 0

def is_power(value):
    if value == "power plant / renewable energy":
        return 1
    else:
        return 0

# "business_area" 열에서 함수를 적용하여 새로운 열 생성
df_train['is_hospital'] = df_train['business_area'].apply(is_hospital)
df_train['is_power'] = df_train['business_area'].apply(is_hospital)
df_test['is_hospital'] = df_test['business_area'].apply(is_hospital)
df_test['is_power'] = df_test['business_area'].apply(is_hospital)

df_train.drop(columns=["business_area"], inplace=True)
df_test.drop(columns=["business_area"], inplace=True)

In [58]:
'''
# expected_timeline 에서 특정 단어 필터링
'''
# 새로운 열을 추가하기 위한 함수 정의
def contains_budget(value):
    if 'budget' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_budget'] = df_train['expected_timeline'].apply(contains_budget)
df_test['contains_budget'] = df_test['expected_timeline'].apply(contains_budget)

# 새로운 열을 추가하기 위한 함수 정의
def contains_etc(value):
    if 'etc' in str(value).lower():  # 대소문자 구분없이 'etc'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_etc'] = df_train['expected_timeline'].apply(contains_etc)
df_test['contains_etc'] = df_test['expected_timeline'].apply(contains_etc)

# 새로운 열을 추가하기 위한 함수 정의
def contains_hence(value):
    if 'hence' in str(value).lower():  # 대소문자 구분없이 'hence'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_hence'] = df_train['expected_timeline'].apply(contains_hence)
df_test['contains_hence'] = df_test['expected_timeline'].apply(contains_hence)

# 새로운 열을 추가하기 위한 함수 정의
def contains_system(value):
    if 'system' in str(value).lower():  # 대소문자 구분없이 'system'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_system'] = df_train['expected_timeline'].apply(contains_system)
df_test['contains_system'] = df_test['expected_timeline'].apply(contains_system)

# 새로운 열을 추가하기 위한 함수 정의
def contains_closi(value):
    if 'closi' in str(value).lower():  # 대소문자 구분없이 'closi'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_closi'] = df_train['expected_timeline'].apply(contains_closi)
df_test['contains_closi'] = df_test['expected_timeline'].apply(contains_closi)

# 새로운 열을 추가하기 위한 함수 정의
def contains_any(value):
    if 'any' in str(value).lower():  # 대소문자 구분없이 'any'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_any'] = df_train['expected_timeline'].apply(contains_any)
df_test['contains_any'] = df_test['expected_timeline'].apply(contains_any)

# 새로운 열을 추가하기 위한 함수 정의
def contains_although(value):
    if 'although' in str(value).lower():  # 대소문자 구분없이 'although'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_although'] = df_train['expected_timeline'].apply(contains_although)
df_test['contains_although'] = df_test['expected_timeline'].apply(contains_although)

# 새로운 열을 추가하기 위한 함수 정의
def contains_more(value):
    if 'more' in str(value).lower():  # 대소문자 구분없이 'more'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_more'] = df_train['expected_timeline'].apply(contains_more)
df_test['contains_more'] = df_test['expected_timeline'].apply(contains_more)

# 새로운 열을 추가하기 위한 함수 정의
def contains_year(value):
    if 'year' in str(value).lower():  # 대소문자 구분없이 'year'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_year'] = df_train['expected_timeline'].apply(contains_year)
df_test['contains_year'] = df_test['expected_timeline'].apply(contains_year)

In [59]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)
x_train

,bant_submit,business_unit,customer_idx,customer_type,enterprise,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,inquiry_type,...,is_power,contains_budget,contains_etc,contains_hence,contains_system,contains_closi,contains_any,contains_although,contains_more,contains_year
43573,0.50,AS,5293,NaN,Enterprise,NaN,NaN,NaN,business development,Sales Inquiry,...,0,0,0,0,0,0,0,0,0,0
25857,0.75,ID,11045,End-Customer,Enterprise,NaN,NaN,NaN,marketing,Quotation or Purchase Consultation,...,0,0,0,0,0,0,0,0,0,0
42264,0.25,AS,47466,NaN,Enterprise,NaN,NaN,NaN,NaN,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0
7665,1.00,ID,12526,NaN,Enterprise,NaN,NaN,NaN,NaN,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0
23088,1.00,ID,40086,NaN,Enterprise,NaN,NaN,NaN,NaN,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,1.00,AS,39568,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,engineering,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0
46009,0.50,AS,35689,NaN,Enterprise,NaN,NaN,NaN,other,Sales Inquiry,...,0,0,0,0,0,0,0,0,0,0
2366,0.75,ID,25096,NaN,Enterprise,NaN,NaN,NaN,NaN,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0
3535,0.50,ID,5386,NaN,Enterprise,NaN,NaN,NaN,business development,Quotation or purchase consultation,...,0,0,0,0,0,0,0,0,0,0


In [60]:
cat_features_col = list(x_train.columns)
cat_features_col.remove("bant_submit")
cat_features_col.remove("com_reg_ver_win_rate_standardized")
cat_features_col.remove("historical_existing_cnt_standardized")
cat_features_col.remove("id_strategic_ver")
cat_features_col.remove("it_strategic_ver")
cat_features_col.remove("idit_strategic_ver")
cat_features_col.remove("lead_desc_length_standardized")
cat_features_col.remove("ver_cus")
cat_features_col.remove("ver_pro")
cat_features_col.remove("ver_win_rate_x_normalized")
cat_features_col.remove("ver_win_ratio_per_bu_normalized")

In [61]:
'''
# 결측치 최빈값으로 채우고 모델 돌리기
'''
# Calculate the most frequent value (mode) for each column
modes = x_train.mode().iloc[0]

# Fill missing (NA) values with the mode
x_train.fillna(modes, inplace=True)

catboost_model = CatBoostClassifier(
    verbose=0, 
    iterations=1000, 
    depth=6, 
    learning_rate=0.1, 
    loss_function='Logloss', 
    one_hot_max_size=5,
    cat_features=cat_features_col
    )

In [62]:
catboost_model.fit(x_train, y_train)

In [63]:
# 모델을 사용하여 검증 데이터셋에 대한 예측 수행
# Calculate the most frequent value (mode) for each column
modes = x_val.mode().iloc[0]

# Fill missing (NA) values with the mode
x_val.fillna(modes, inplace=True)

y_val_pred = catboost_model.predict(x_val)

y_val_pred = np.array([True if pred == 'True' else False for pred in y_val_pred])

print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.99      0.99     10913
        True       0.93      0.79      0.85       947

    accuracy                           0.98     11860
   macro avg       0.96      0.89      0.92     11860
weighted avg       0.98      0.98      0.98     11860

오차행렬:
 [[  749   198]
 [   58 10855]]

정확도: 0.9784
정밀도: 0.9281
재현율: 0.7909
F1: 0.8540


In [64]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

# Calculate the most frequent value (mode) for each column
modes = x_test.mode().iloc[0]

# Fill missing (NA) values with the mode
x_test.fillna(modes, inplace=True)

test_pred = catboost_model.predict(x_test)

df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)